# ~ Category Arrangement Task Analysis ~

In [3]:
import os, glob
import pandas as pd
from scipy.io import loadmat
import numpy as np
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA
from scipy.spatial.distance import squareform
import seaborn as sns

from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection

from sklearn import manifold
from sklearn.metrics import euclidean_distances
from sklearn.decomposition import PCA

In [466]:
work_dir = '/Volumes/GoogleDrive/My Drive/categories_dimensions/subject_data'
os.chdir(work_dir)

## Import category arrangement data

In [460]:
subjects_data = []
for file in glob.glob("*.csv"):
    subjects_data.append(file)  

In [502]:
subjects_data

['divine-pigeon_soc-690.csv']

In [501]:
#Importing some multi-arrangment data to obtain same relationships variable 
mat_subj = []
mat_subj2 = []
for file in glob.glob('*.mat'):
    mat_subj.append(file)
for j in range(48):
    mat = loadmat(mat_subj[j])
    mat_subj2.append(mat)

#Delete excess spacing for relationship values
relationships=[]
for i in mat['stimuli']:
    delete_spaces = i.replace('  ','')
    delete_spaces = delete_spaces.strip()
    relationships.append(delete_spaces)

In [447]:
#csv_subj = pd.read_csv('Meadows_Social_Relationship_Knowledge_v_v2_amazed-cub_2_table.csv').iloc[:, 1:]

r1 = relationships

In [485]:
soc_id = pd.read_csv('/Volumes/GoogleDrive/My Drive/categories_dimensions/soc_meadows.csv')
soc_meadows = soc_id.drop(['Unnamed: 0'], axis = 1)
#soc_meadows = pd.concat([soc_id['id'], soc_id['meadows_id']], axis=1, keys=['soc_id', 'meadows_id'])
#soc_meadows


## Create RDM of relationship categories for 1 subject

In [487]:
#Creates a dataframe of soc_id and meadows_id for each subject
soc_meadows.drop(soc_meadows.index[51:], inplace = True)
soc_meadows

,soc_id,meadows_id
0,soc-690,divine-pigeon
1,soc-691,amazed-cub
2,soc-692,ready-sloth
3,soc-651,whole-mantis
4,soc-320,meet-mule
5,soc-693,enough-falcon
6,soc-694,more-ferret
7,soc-695,neat-marten
8,soc-696,keen-hen
9,soc-697,moral-cub


In [ ]:
#This for loop saves each dataframe output into a dictionary that is accesible by a string of meadows id
#It can also be used to save each subject's RDM into a separate file with meadows id and soc id 

rdm_r1 = pd.DataFrame(columns=relationships)
meadows_list = []
dict_rdms = {}
for subject in subjects_data:
    #csv_subj = pd.read_csv(subject).iloc[:, 1:]
    rdm_r1 = pd.DataFrame(columns=relationships)
    for r1_a in relationships:
        cols = [] 
        for col in csv_subj.columns:
            if r1_a in csv_subj[col].tolist():
                cols.append(col)
        for r1_b in relationships:
            if not cols:
                rdm_r1.loc[r1_a,r1_b] = 0
            elif csv_subj[cols].isin([r1_b]).any()[0]:
                rdm_r1.loc[r1_a,r1_b] = 1
            else:
                rdm_r1.loc[r1_a,r1_b] = 0
    meadows_list = []
    meadows_id = subject.split('_')[6]
    meadows_list.append(meadows_id)
    meadows_list = pd.DataFrame(meadows_list)    
    meadows_list.columns = ['meadows_id']
    meadows_list.set_index('meadows_id', inplace=True)
    merged_df = meadows_list.merge(soc_meadows, how = 'inner', on = ['meadows_id', 'meadows_id'])
    dict_rdms[meadows_id] = rdm_r1
    #The code below can be uncommented to save each dataframe as a file into folder 
    #work_dir3 = '/Volumes/GoogleDrive/My Drive/categories_dimensions/subject_data/Subject_Category_RDM'
    #os.chdir(work_dir3)
    #rdm_r1.to_csv('{}.csv'.format((merged_df['meadows_id'][0]+'_'+merged_df['soc_id'][0])), sep=',')
    #print('next')
                  

#Access a specific RDM by printing d[*insert 'meadows_id' here*]    


## Create RDM for all subjects

In [10]:
#Turn subject matrices to 1D arrays so they can be correlated
subject_flat_arrays = []
subjects_rdm = pd.DataFrame(columns=subject_cat_frames.keys())
for subj_a in subject_cat_frames.keys():
    subj_a_flat = subject_cat_frames[subj_a].values.flatten()
    subject_flat_arrays.append(subj_a_flat)
    for subj_b in subject_cat_frames.keys():
        subj_b_flat = subject_cat_frames[subj_b].values.flatten()
        subjects_rdm.loc[subj_a,subj_b] = np.corrcoef(subj_a_flat, subj_b_flat)[0][1]



## Calculate how often each of the relationships are grouped together

In [73]:
rel_gr_together = subject_cat_frames[list(subject_cat_frames)[0]]
for subj in list(subject_cat_frames)[1:]:
    #rel_gr_together.add(subject_cat_frames[subj_a], fill_value=0)
    rel_gr_together = rel_gr_together + subject_cat_frames[subj]
rel_gr_together = rel_gr_together.apply(pd.to_numeric)
rel_gr_together = rel_gr_together.div(len(subject_cat_frames))

In [74]:
rel_gr_together

,Wife - Husband,Nurse - Patient,Officer - Soldier,Neighbors,Athletic trainer - Trainee,Police officer - Offender,Principal - Teacher,Doctor - Nurse,Victim - Criminal,Coach - Athlete,...,Sorority sisters,Classmates,Club-member - Club-president,Political opponents,Friends-with-benefits,Foster-parent - Foster-child,Business rivals,Manager - Assistant,Victim - Witness,Business partners
Wife - Husband,0.000000,1.000000,1.000000,0.916667,1.000000,1.000000,1.000000,0.979167,1.000000,1.000000,...,0.812500,0.937500,1.000000,1.000000,0.458333,0.625000,1.000000,1.000000,0.979167,0.937500
Nurse - Patient,1.000000,0.000000,0.770833,0.937500,0.562500,0.875000,0.770833,0.583333,0.958333,0.750000,...,1.000000,0.937500,0.791667,0.958333,0.979167,0.916667,0.937500,0.812500,0.854167,0.833333
Officer - Soldier,0.979167,0.750000,0.000000,1.000000,0.604167,0.604167,0.250000,0.333333,0.770833,0.583333,...,1.000000,1.000000,0.625000,0.937500,1.000000,0.916667,0.895833,0.354167,0.833333,0.770833
Neighbors,0.895833,0.958333,0.979167,0.000000,0.958333,0.979167,0.958333,0.958333,0.958333,0.916667,...,0.520833,0.375000,0.833333,0.875000,0.729167,0.958333,0.875000,0.979167,0.750000,0.750000
Athletic trainer - Trainee,1.000000,0.562500,0.645833,0.937500,0.000000,0.895833,0.625000,0.583333,0.958333,0.375000,...,0.916667,0.958333,0.687500,0.916667,0.979167,0.979167,0.854167,0.750000,0.895833,0.770833
Police officer - Offender,0.979167,0.854167,0.625000,0.979167,0.875000,0.000000,0.791667,0.812500,0.229167,0.791667,...,1.000000,0.979167,0.833333,0.500000,0.979167,0.937500,0.583333,0.750000,0.750000,0.958333
Principal - Teacher,1.000000,0.770833,0.270833,0.958333,0.604167,0.791667,0.000000,0.270833,0.958333,0.541667,...,0.979167,0.895833,0.645833,0.937500,0.979167,0.958333,0.875000,0.416667,0.916667,0.729167
Doctor - Nurse,0.979167,0.625000,0.375000,0.958333,0.604167,0.812500,0.291667,0.000000,0.937500,0.625000,...,0.958333,0.916667,0.708333,0.937500,0.979167,0.958333,0.875000,0.541667,0.854167,0.583333
Victim - Criminal,0.979167,0.958333,0.791667,0.958333,0.958333,0.229167,0.958333,0.937500,0.000000,0.916667,...,1.000000,0.958333,0.916667,0.395833,0.979167,0.958333,0.479167,0.895833,0.645833,0.979167
Coach - Athlete,0.958333,0.750000,0.604167,0.916667,0.354167,0.791667,0.541667,0.604167,0.916667,0.000000,...,0.916667,0.916667,0.729167,0.875000,0.979167,0.937500,0.854167,0.666667,0.937500,0.833333


## View the representational dissimilarity matrix between all subjects

# ** Do not run cell below **

In [67]:
subjects_rdm

,subj_1,subj_2,subj_3,subj_4,subj_5,subj_6,subj_7,subj_8,subj_9,subj_10,...,subj_39,subj_40,subj_41,subj_42,subj_43,subj_44,subj_45,subj_46,subj_47,subj_48
subj_1,1,0.168303,0.242884,0.246024,0.2723,0.172421,0.21229,0.324263,0.2546,0.244418,...,0.285776,0.340895,0.303615,0.407036,0.21707,0.248502,0.279642,0.229904,0.335239,0.302243
subj_2,0.168303,1,0.159223,0.192502,0.135599,0.144967,0.157873,0.198326,0.146327,0.160177,...,0.176598,0.171434,0.178687,0.153038,0.137423,0.230435,0.165743,0.184932,0.211252,0.160767
subj_3,0.242884,0.159223,1,0.187592,0.16867,0.151554,0.152489,0.34359,0.334086,0.160372,...,0.235343,0.237609,0.261537,0.366442,0.224492,0.150908,0.303763,0.205441,0.376515,0.244933
subj_4,0.246024,0.192502,0.187592,1,0.232082,0.152954,0.331607,0.204017,0.244584,0.32852,...,0.351189,0.299642,0.274654,0.293909,0.301997,0.55876,0.29449,0.214769,0.324609,0.264037
subj_5,0.2723,0.135599,0.16867,0.232082,1,0.125976,0.238339,0.242811,0.222721,0.185606,...,0.235687,0.306097,0.238232,0.327629,0.221398,0.240969,0.232274,0.210864,0.259862,0.297154
subj_6,0.172421,0.144967,0.151554,0.152954,0.125976,1,0.246337,0.193629,0.196863,0.208003,...,0.182018,0.293926,0.271069,0.188943,0.208981,0.18158,0.224423,0.200392,0.168744,0.247403
subj_7,0.21229,0.157873,0.152489,0.331607,0.238339,0.246337,1,0.164832,0.202723,0.355565,...,0.281778,0.33324,0.240955,0.246938,0.230605,0.380997,0.221481,0.240785,0.243598,0.287665
subj_8,0.324263,0.198326,0.34359,0.204017,0.242811,0.193629,0.164832,1,0.351744,0.20042,...,0.278388,0.251398,0.366291,0.459288,0.245381,0.180478,0.303674,0.227929,0.342008,0.277311
subj_9,0.2546,0.146327,0.334086,0.244584,0.222721,0.196863,0.202723,0.351744,1,0.208341,...,0.279897,0.280884,0.320989,0.40185,0.311785,0.208815,0.249605,0.286463,0.381736,0.370296
subj_10,0.244418,0.160177,0.160372,0.32852,0.185606,0.208003,0.355565,0.20042,0.208341,1,...,0.312009,0.309105,0.281719,0.255529,0.310995,0.341002,0.277935,0.169515,0.239971,0.260946


In [245]:
subjects_rdm.mean(axis = 0) 

#Remove subjects with avg RDM correlation less than 0.2 and avg category number less than 2 SDs from the mean 
subjects_rdm.drop(['subj_3', 'subj_15', 'subj_26', 'subj_45'], axis = 1)
subjects_rdm.drop(['subj_3', 'subj_15', 'subj_26', 'subj_45'], axis = 0)


,subj_1,subj_2,subj_3,subj_4,subj_5,subj_6,subj_7,subj_8,subj_9,subj_10,...,subj_40,subj_41,subj_42,subj_43,subj_44,subj_45,subj_46,subj_47,subj_48,subj_49
subj_1,1,0.355565,0.157873,0.34517,0.230605,0.221481,0.238476,0.243598,0.202723,0.186302,...,0.277157,0.143894,0.21229,0.287665,0.240955,0.274843,0.162454,0.380997,0.192011,0.321731
subj_2,0.355565,1,0.160177,0.302923,0.310995,0.277935,0.22949,0.239971,0.208341,0.228882,...,0.27362,0.161965,0.244418,0.260946,0.281719,0.254063,0.0749729,0.341002,0.217372,0.332402
subj_4,0.34517,0.302923,0.165444,1,0.321736,0.337253,0.378801,0.351277,0.272115,0.285378,...,0.47616,0.192735,0.384568,0.403348,0.38814,0.29644,0.197067,0.313236,0.34465,0.355814
subj_5,0.230605,0.310995,0.137423,0.321736,1,0.281386,0.269654,0.326221,0.311785,0.213487,...,0.288059,0.151075,0.21707,0.307661,0.308562,0.283581,0.108406,0.283459,0.227099,0.402018
subj_6,0.221481,0.277935,0.165743,0.337253,0.281386,1,0.200917,0.218556,0.249605,0.217752,...,0.304722,0.239205,0.279642,0.234006,0.352303,0.279434,0.292141,0.254291,0.318457,0.291082
subj_7,0.238476,0.22949,0.18203,0.378801,0.269654,0.200917,1,0.442505,0.347078,0.248143,...,0.375402,0.219029,0.33674,0.408243,0.24131,0.267001,0.162983,0.241513,0.161632,0.364451
subj_8,0.243598,0.239971,0.211252,0.351277,0.326221,0.218556,0.442505,1,0.381736,0.220461,...,0.359718,0.272565,0.335239,0.385869,0.256473,0.249651,0.174613,0.311041,0.174338,0.385582
subj_9,0.202723,0.208341,0.146327,0.272115,0.311785,0.249605,0.347078,0.381736,1,0.219611,...,0.285732,0.26967,0.2546,0.370296,0.320989,0.306628,0.285242,0.208815,0.185472,0.447383
subj_10,0.186302,0.228882,0.178262,0.285378,0.213487,0.217752,0.248143,0.220461,0.219611,1,...,0.280883,0.237184,0.274093,0.24934,0.245644,0.244431,0.0979704,0.237425,0.261805,0.258581
subj_11,0.285493,0.277558,0.191117,0.493521,0.258554,0.303175,0.370361,0.427091,0.34137,0.264377,...,0.431155,0.233585,0.346448,0.425124,0.382612,0.284994,0.234119,0.291845,0.2662,0.407309


In [250]:
pd.set_option('display.max_colwidth', -1)
li = []
li.append([])
li.append([])
li
#Create a function to return the number of correlations each subject has that are greater than 0.3
def find_values(subjects_rdm):
    count = 0
    for row in subjects_rdm:
        rslt_df = [subjects_rdm[row] >= 0.2]
        k = rslt_df[row==True].value_counts()
        li.append(k)
        k = pd.DataFrame(k)
        print(k)
        #maxk = k.max 
        #mink = k.min
        

#Create a dataframe listing the matching subjet
df_subjs_matching = subjects_rdm.apply(lambda x: ','.join(x.index[x >= 0.3]), axis=1)
df_subjs_matching = pd.DataFrame(df_subjs_matching)
        
find_values(subjects_rdm)
df_subjs_matching


       subj_1
True   36    
False  13    
       subj_2
True   37    
False  12    
       subj_3
False  43    
True   6     
       subj_4
True   43    
False  6     
       subj_5
True   42    
False  7     
       subj_6
True   44    
False  5     
       subj_7
True   41    
False  8     
       subj_8
True   42    
False  7     
       subj_9
True   44    
False  5     
       subj_10
True   38     
False  11     
       subj_11
True   45     
False  4      
       subj_12
True   45     
False  4      
       subj_13
True   46     
False  3      
       subj_14
True   35     
False  14     
       subj_15
False  43     
True   6      
       subj_16
True   37     
False  12     
       subj_17
True   39     
False  10     
       subj_18
True   42     
False  7      
       subj_19
True   44     
False  5      
       subj_20
True   46     
False  3      
       subj_21
False  32     
True   17     
       subj_22
True   39     
False  10     
       subj_23
True   45     
False  

,0
subj_1,"subj_1,subj_2,subj_4,subj_12,subj_17,subj_18,subj_20,subj_24,subj_25,subj_35,subj_37,subj_47,subj_49"
subj_2,"subj_1,subj_2,subj_4,subj_5,subj_12,subj_17,subj_18,subj_20,subj_27,subj_35,subj_37,subj_47,subj_49"
subj_3,subj_3
subj_4,"subj_1,subj_2,subj_4,subj_5,subj_6,subj_7,subj_8,subj_11,subj_12,subj_13,subj_14,subj_17,subj_18,subj_19,subj_20,subj_23,subj_24,subj_25,subj_32,subj_33,subj_35,subj_37,subj_39,subj_40,subj_42,subj_43,subj_44,subj_47,subj_48,subj_49"
subj_5,"subj_2,subj_4,subj_5,subj_8,subj_9,subj_12,subj_13,subj_17,subj_18,subj_22,subj_27,subj_32,subj_37,subj_43,subj_44,subj_49"
subj_6,"subj_4,subj_6,subj_11,subj_12,subj_13,subj_18,subj_20,subj_23,subj_28,subj_29,subj_30,subj_35,subj_36,subj_37,subj_39,subj_40,subj_44,subj_48"
subj_7,"subj_4,subj_7,subj_8,subj_9,subj_11,subj_13,subj_19,subj_20,subj_22,subj_24,subj_25,subj_27,subj_28,subj_29,subj_32,subj_33,subj_35,subj_37,subj_39,subj_40,subj_42,subj_43,subj_49"
subj_8,"subj_4,subj_5,subj_7,subj_8,subj_9,subj_11,subj_13,subj_17,subj_18,subj_19,subj_20,subj_22,subj_24,subj_25,subj_27,subj_28,subj_29,subj_32,subj_33,subj_35,subj_36,subj_37,subj_40,subj_42,subj_43,subj_47,subj_49"
subj_9,"subj_5,subj_7,subj_8,subj_9,subj_11,subj_13,subj_20,subj_28,subj_32,subj_33,subj_35,subj_36,subj_43,subj_44,subj_45,subj_49"
subj_10,"subj_10,subj_13,subj_18,subj_39"
